In [ ]:
! pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
dataset_id="facebook/voxpopuli"

In [ ]:
def get_subset(ds, size):
    subset=[]
    for i, example in enumerate(ds):
        if i >= size:
            break
        subset.append(example)
    return subset

In [ ]:
en_ds = load_dataset(dataset_id, "en", split="train", streaming=True)
ro_ds = load_dataset(dataset_id, "ro", split="train", streaming=True)
sp_ds = load_dataset(dataset_id, "es", split="train", streaming=True)

voxpopuli.py:   0%|          | 0.00/8.84k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

The repository for facebook/voxpopuli contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/voxpopuli.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
size=50
en_sub=get_subset(en_ds, size)
ro_sub=get_subset(ro_ds, size)
sp_sub=get_subset(sp_ds, size)

In [ ]:
subset=[]
subset.extend(en_sub)
subset.extend(sp_sub)
subset.extend(ro_sub)

In [ ]:
len(subset)

150

In [ ]:
# total seconds
len(subset[0]["audio"]["array"])/subset[0]["audio"]["sampling_rate"]

8.3999375

In [ ]:
subset[0]

{'audio_id': '20180418-0900-PLENARY-3-en_20180418-08:50:36_17',
 'language': 0,
 'audio': {'path': 'train_part_0/20180418-0900-PLENARY-3-en_20180418-08:50:36_17.wav',
  'array': array([-0.00030518,  0.00119019,  0.00506592, ..., -0.00036621,
         -0.00027466, -0.00018311]),
  'sampling_rate': 16000},
 'raw_text': 'If you do not address this problem, the ground is there for populist nationalist forces to go on growing all over Europe.',
 'normalized_text': 'if you do not address this problem the ground is there for populist nationalist forces to go on growing all over europe.',
 'gender': 'female',
 'speaker_id': '124737',
 'is_gold_transcript': True,
 'accent': 'None'}

In [ ]:
subset[101]

{'audio_id': '20110404-0900-PLENARY-12-ro_20110404-20:10:42_1',
 'language': 6,
 'audio': {'path': 'train_part_0/20110404-0900-PLENARY-12-ro_20110404-20:10:42_1.wav',
  'array': array([-0.00344849, -0.00927734, -0.0005188 , ..., -0.01364136,
         -0.01565552, -0.01293945]),
  'sampling_rate': 16000},
 'raw_text': 'Există zone pe glob unde violența contra femeilor se manifestă sub toate formele viol, trafic sexual, căsătorii forțate, moarte, răpiri din motive religioase sau criminale, prostituție forțată, la care se adaugă violența în familie sau din partea foștilor parteneri.',
 'normalized_text': 'există zone pe glob unde violența contra femeilor se manifestă sub toate formele viol trafic sexual căsătorii forțate moarte răpiri din motive religioase sau criminale prostituție forțată la care se adaugă violența în familie sau din partea foștilor parteneri.',
 'gender': 'female',
 'speaker_id': '95281',
 'is_gold_transcript': True,
 'accent': 'None'}

In [ ]:
import random
random.shuffle(subset)

In [ ]:
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from tqdm import tqdm

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
# Load the Wav2Vec processor and model
model_id="facebook/wav2vec2-base-960h"
processor=Wav2Vec2Processor.from_pretrained(model_id)
model=Wav2Vec2Model.from_pretrained(model_id)

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
call_count=0
def extract_wav2vec_features(batch):
    global call_count
    call_count+=1
    print(call_count)
    inputs=processor(batch["audio"]["array"], sampling_rate=16000,return_tensors="pt",padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return {"features": outputs.last_hidden_state.squeeze().mean(dim=0).numpy(), "label":batch["language"]}

The mysterious last hidden state: https://io.traffine.com/en/articles/last-hidden-state

In [ ]:
# Extract Wav2Vec features for the dataset

subset_with_features = [extract_wav2vec_features(example) for example in subset]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# split the dataset into train, validation and test set

train_data, test_data = train_test_split(subset_with_features, test_size=0.2)
train_data, val_data = train_test_split(train_data, test_size=0.1)

# extract features and labels

def prepare_data(data):
    features = [item["features"] for item in data]
    labels = [item["label"] for item in data]
    return torch.tensor(features), torch.tensor(labels)

train_features, train_labels = prepare_data(train_data)
val_features, val_labels = prepare_data(val_data)
test_features, test_labels = prepare_data(test_data)

<ipython-input-18-d1f0dcecbf1a>:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(features), torch.tensor(labels)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Let's build the model

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=128, kernel_size=3,padding=1)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3,padding=1)
        self.fc1 = nn.Linear(256 * (input_length//4), 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool1d(x, kernel_size=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool1d(x, kernel_size=2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# assuming train_features and val features are already loaded
# and they have the shape [batch_size, sequence_length]

input_length = train_features.shape[1] # Add this line to get the sequence length

label_mapping = {0:0, 3:1, 6:2} # en, es, ro
train_labels = train_labels.apply_(lambda x: label_mapping[x])
val_labels = val_labels.apply_(lambda x: label_mapping[x])
test_labels = test_labels .apply_(lambda x: label_mapping[x])

unique_train_labels = torch.unique(train_labels)
unique_val_labels = torch.unique(val_labels)
unique_test_labels = torch.unique(test_labels)

print(f"Mapped unique train labels: {unique_train_labels}")
print(f"Mapped unique val labels: {unique_val_labels}")
print(f"Mapped unique test labels: {unique_test_labels}")

Mapped unique train labels: tensor([0, 1, 2])
Mapped unique val labels: tensor([0, 1, 2])
Mapped unique test labels: tensor([0, 1, 2])


In [ ]:
model = CNNModel(input_dim=1, num_classes=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs=20

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(train_features.unsqueeze(1).float())
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()

    # validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(val_features.unsqueeze(1).float())
        val_loss = criterion(val_outputs, val_labels)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")

Epoch 1/20, Loss: 1.0985544919967651, Val Loss: 2.235348701477051
Epoch 2/20, Loss: 2.2655582427978516, Val Loss: 1.3017269372940063
Epoch 3/20, Loss: 1.1821198463439941, Val Loss: 1.2742652893066406
Epoch 4/20, Loss: 1.224250078201294, Val Loss: 1.1180791854858398
Epoch 5/20, Loss: 1.1044844388961792, Val Loss: 1.0660449266433716
Epoch 6/20, Loss: 1.0692628622055054, Val Loss: 1.0522921085357666
Epoch 7/20, Loss: 1.054453730583191, Val Loss: 1.0391061305999756
Epoch 8/20, Loss: 1.0427584648132324, Val Loss: 0.9911778569221497
Epoch 9/20, Loss: 1.0127463340759277, Val Loss: 0.9458208680152893
Epoch 10/20, Loss: 0.9654945135116577, Val Loss: 0.916614294052124
Epoch 11/20, Loss: 0.922095537185669, Val Loss: 0.8717966079711914
Epoch 12/20, Loss: 0.8708404898643494, Val Loss: 0.8029093742370605
Epoch 13/20, Loss: 0.8177242279052734, Val Loss: 0.8071714043617249
Epoch 14/20, Loss: 0.7670100927352905, Val Loss: 0.700223445892334
Epoch 15/20, Loss: 0.7532158493995667, Val Loss: 0.748598575592

In [ ]:
import torch
from sklearn.metrics import accuracy_score

In [ ]:
# model in evaluation mode

model.eval()

with torch.no_grad():
    # forward pass to get predictions
    test_outputs = model(test_features.unsqueeze(1).float())

    # get the predicted class labels
    _, predicted = torch.max(test_outputs, 1)

    # calculate accuracy
    accuracy = accuracy_score(test_labels.numpy(), predicted.numpy())

print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 73.33%


eference

#  Reference Notebooks

[[0] Wav2Vec2](https://huggingface.co/docs/transformers/model_doc/wav2vec2)

* Automatic Speech Recognition
* Audio Classification
* Deploy

[[1] Fine-Tune Speech Recognition Model with 🤗 Transformers using CTC.ipynb](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/speech_recognition.ipynb#scrollTo=tborvC9hx88e)


[[2] Emotion recognition in Greek speech using Wav2Vec2.ipynb](https://colab.research.google.com/github/m3hrdadfi/soxan/blob/main/notebooks/Emotion_recognition_in_Greek_speech_using_Wav2Vec2.ipynb#scrollTo=BkEd4w8IV7kZ)


[[3] Audio Classification](https://huggingface.co/docs/transformers/tasks/audio_classification)

[[4] HF Notebooks](https://huggingface.co/docs/transformers/v4.45.2/en/notebooks)